In [3]:
!pip install pandas scikit-learn numpy

import pandas as pd
import os
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

  Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached scipy-1.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.3 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)


In [5]:
print("Speaker Attribution Analysis Script")
print("----------------------------------")

# Load ground truth data
ground_truth = pd.read_csv("input/quote_info.csv")
print(f"Loaded ground truth with {len(ground_truth)} quotes")

# Load model outputs
small_model = pd.read_csv("output_small/novel_small.quotes", sep="\t")
print(f"Loaded small model output with {len(small_model)} quotes")

big_model = pd.read_csv("output_big/novel_big.quotes", sep="\t")
print(f"Loaded big model output with {len(big_model)} quotes")


# Calculate accuracy
def calculate_accuracy(model, ground_truth):
    correct = sum(model["char_id"] == ground_truth["char_id"])
    total = len(ground_truth)
    return correct / total


small_accuracy = calculate_accuracy(small_model, ground_truth)
big_accuracy = calculate_accuracy(big_model, ground_truth)

print(f"\nAccuracy Results:")
print(f"Small model accuracy: {small_accuracy:.2%}")
print(f"Big model accuracy: {big_accuracy:.2%}")


# Find misattributions
def find_errors(model, ground_truth):
    errors = []
    for i, (_, gt_row) in enumerate(ground_truth.iterrows()):
        model_row = model.iloc[i]
        if gt_row["char_id"] != model_row["char_id"]:
            errors.append({
                "quote_id": gt_row["quote_id"],
                "quote_text": gt_row["quote_text"],
                "true_speaker": gt_row["speaker"],
                "predicted_speaker": model_row["speaker"],
                "true_id": gt_row["char_id"],
                "predicted_id": model_row["char_id"]
            })
    return errors


small_errors = find_errors(small_model, ground_truth)
big_errors = find_errors(big_model, ground_truth)

print(f"\nSmall Model Errors ({len(small_errors)}):")
for error in small_errors:
    print(f"Quote {error['quote_id']}: {error['quote_text']}")
    print(f"  True speaker: {error['true_speaker']}")
    print(f"  Predicted: {error['predicted_speaker']}")
    print()

print(f"\nBig Model Errors ({len(big_errors)}):")
for error in big_errors:
    print(f"Quote {error['quote_id']}: {error['quote_text']}")
    print(f"  True speaker: {error['true_speaker']}")
    print(f"  Predicted: {error['predicted_speaker']}")
    print()

# Error analysis by speaker
speakers = ground_truth["speaker"].unique()
print("\nError Analysis by Speaker:")
for speaker in speakers:
    speaker_quotes = ground_truth[ground_truth["speaker"] == speaker]
    small_speaker_errors = sum(small_model.iloc[speaker_quotes.index]["speaker"] != speaker)
    big_speaker_errors = sum(big_model.iloc[speaker_quotes.index]["speaker"] != speaker)

    total = len(speaker_quotes)
    print(f"{speaker}: {total} quotes")
    print(f"  Small model errors: {small_speaker_errors}/{total} ({small_speaker_errors / total:.2%})")
    print(f"  Big model errors: {big_speaker_errors}/{total} ({big_speaker_errors / total:.2%})")

print("\nAnalysis complete!")

Speaker Attribution Analysis Script
----------------------------------
Loaded ground truth with 18 quotes
Loaded small model output with 18 quotes
Loaded big model output with 18 quotes

Accuracy Results:
Small model accuracy: 88.89%
Big model accuracy: 94.44%

Small Model Errors (2):
Quote 5: "Charlotte, don't you feel, too, that we might be in London? I can hardly believe that all kinds of other things are just outside. I suppose it is one's being so tired."
  True speaker: Lucy
  Predicted: Miss Bartlett

Quote 14: "I have a view, I have a view."
  True speaker: Old Man
  Predicted: Unknown


Big Model Errors (1):
Quote 17: "his name's George. He has a view too."
  True speaker: Old Man
  Predicted: Lucy


Error Analysis by Speaker:
Miss Bartlett: 10 quotes
  Small model errors: 0/10 (0.00%)
  Big model errors: 0/10 (0.00%)
Lucy: 5 quotes
  Small model errors: 1/5 (20.00%)
  Big model errors: 0/5 (0.00%)
Old Man: 3 quotes
  Small model errors: 1/3 (33.33%)
  Big model errors: 1/3 (3